# Fine-Tune DistilBERT for News Bias Detection

This notebook demonstrates how to fine-tune the `distilbert-base-uncased` transformer model to detect bias in news articles using the AllSides, MBFC, and BASIL datasets. The workflow is designed for Google Colab to leverage free GPU resources.

## 1. Set Up Google Colab Environment

Make sure you are running this notebook on Google Colab with GPU acceleration enabled. Go to `Runtime > Change runtime type` and select `GPU`.

## 2. Install Required Libraries

Install Hugging Face Transformers, Datasets, and other dependencies.

In [ ]:
# Install required libraries
!pip install transformers datasets pandas numpy scikit-learn torch nlpaug sacremoses --quiet

**Description:** This cell installs all required Python libraries for model training and data processing, including Hugging Face Transformers, Datasets, pandas, numpy, scikit-learn, and torch.

## 3. Import Libraries

Import all necessary Python libraries for data processing and model training.

In [4]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
from nlpaug.util import Action

**Description:** This cell provides instructions and sample code to upload or download the AllSides, MBFC, and BASIL datasets, and loads them into pandas DataFrames for further processing.

In [5]:
# Create datasets directory
import os
import zipfile
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

os.makedirs('datasets', exist_ok=True)

datasets = {}

def download_file(url, filename):
    """Download file with progress bar"""
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

    with open(filename, 'wb') as f:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            f.write(data)
    progress_bar.close()

# 1. LIAR Dataset
print("\nDownloading LIAR dataset...")
try:
    liar_url = "https://www.cs.ucsb.edu/~william/data/liar_dataset.zip"
    download_file(liar_url, "datasets/liar.zip")

    with zipfile.ZipFile("datasets/liar.zip", 'r') as zip_ref:
        zip_ref.extractall("datasets/liar")

    # Load LIAR dataset
    liar_train = pd.read_csv('datasets/liar/train.tsv', sep='\t',
                            names=['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                  'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts',
                                  'pants_on_fire_counts', 'context'])
    datasets['liar'] = liar_train
    print("LIAR dataset loaded successfully")
except Exception as e:
    print(f"Error loading LIAR dataset: {str(e)}")

# 2. News Category Dataset
print("\nDownloading News Category Dataset...")
try:
    news_url = "https://raw.githubusercontent.com/rmisra/news-headlines-dataset/master/news_category_dataset_v2.json"
    response = requests.get(news_url)
    with open("datasets/news_category.json", 'w') as f:
        f.write(response.text)

    news_cat_df = pd.read_json('datasets/news_category.json', lines=True)
    datasets['news_category'] = news_cat_df
    print("News Category dataset loaded successfully")
except Exception as e:
    print(f"Error loading News Category dataset: {str(e)}")

# 3. AG News Dataset (as replacement for BABE dataset)
print("\nDownloading AG News Dataset...")
try:
    from datasets import load_dataset
    ag_news = load_dataset("ag_news")
    ag_news_df = pd.DataFrame({
        'text': ag_news['train']['text'],
        'label': ag_news['train']['label']
    })
    datasets['ag_news'] = ag_news_df
    print("AG News dataset loaded successfully")
except Exception as e:
    print(f"Error loading AG News dataset: {str(e)}")

# Clean up zip files
for file in os.listdir('datasets'):
    if file.endswith('.zip'):
        os.remove(os.path.join('datasets', file))

# Print dataset statistics
print("\nDatasets loaded successfully!")
print(f"Total number of samples in each dataset:")
for name, df in datasets.items():
    print(f"{name}: {len(df)} samples")

print("\nSample from each dataset:")
for name, df in datasets.items():
    print(f"\n{name.upper()} sample:")
    print(df.head(1))

100%|██████████| 1.01M/1.01M [00:00<00:00, 6.64MiB/s]


LIAR dataset loaded successfully

Error loading News Category dataset: Unexpected character found when decoding array value (2)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


AG News dataset loaded successfully

Datasets loaded successfully!
Total number of samples in each dataset:
liar: 10240 samples
ag_news: 120000 samples

Sample from each dataset:

LIAR sample:
          id  label                                          statement  \
0  2635.json  false  Says the Annies List political group supports ...   

    subject       speaker                   job  state       party  \
0  abortion  dwayne-bohac  State representative  Texas  republican   

   barely_true_counts  false_counts  half_true_counts  mostly_true_counts  \
0                 0.0           1.0               0.0                 0.0   

   pants_on_fire_counts   context  
0                   0.0  a mailer  

AG_NEWS sample:
                                                text  label
0  Wall St. Bears Claw Back Into the Black (Reute...      2


In [36]:
# Data agumentation

**Description:** This cell cleans and preprocesses the datasets, encodes bias labels, and splits the combined data into training, validation, and test sets for model training.

## 6. Load DistilBERT Model and Tokenizer

Load the `distilbert-base-uncased` model and tokenizer from Hugging Face Transformers.

In [8]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Description:** This cell tokenizes the text data and converts the train, validation, and test sets into Hugging Face Dataset objects, setting them up for efficient batching and training with PyTorch.


# Layer Freezing for DistilBERT Fine-tuning

Dataset Size: working with a relatively small dataset (< 100k examples), freezing layers helps prevent overfitting
Task Similarity: Bias detection is related to the original language understanding task, so the pre-trained weights are valuable
Training Efficiency: Freezing layers reduces training time and memory requirements


# Freezing some layers

Freeze the first 4 layers (more foundational language understanding)
Keep the top 2 layers trainable (task-specific adaptation)
Always keep the classifier layer trainable
Show you how many parameters are frozen vs trainable
This approach provides a good balance between:

Preserving learned language features
Adapting to the specific bias detection task
Training efficiency
Preventing overfitting
The exact number of layers to freeze (4 in this example) can be tuned based on your results.

In [10]:
def freeze_layers(model, num_layers_to_freeze=4):
    """Freeze the first n transformer layers of DistilBERT"""
    # First freeze all parameters
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze layers from top to bottom
    # DistilBERT has 6 layers total
    for i in range(5, num_layers_to_freeze-1, -1):
        for param in model.distilbert.transformer.layer[i].parameters():
            param.requires_grad = True

    # Always unfreeze the classifier layer
    for param in model.classifier.parameters():
        param.requires_grad = True

    # Print trainable parameters info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.1%})")

    return model

# Apply freezing before creating the Trainer
model = freeze_layers(model, num_layers_to_freeze=4)  # Freeze first 4 layers

Total parameters: 66,955,010
Trainable parameters: 14,177,282 (21.2%)


**Description:** This cell sets up the Hugging Face Trainer, defines training arguments, and starts the fine-tuning process for DistilBERT using the prepared datasets. It also defines metrics for model evaluation during training.

In [12]:
print("Starting training...")
trainer.train()

Starting training...


wandb: Currently logged in as: pablowatfi (pablowatfi-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,No log,0.680937,0.664336,0.848485,0.394366,0.538462
100,0.684200,0.648184,0.804196,0.890909,0.690141,0.777778
150,0.684200,0.534439,0.860140,0.869565,0.845070,0.857143
200,0.566300,0.401125,0.888112,0.823529,0.985915,0.897436
250,0.566300,0.348049,0.867133,0.919355,0.802817,0.857143
300,0.388700,0.251488,0.930070,0.906667,0.957746,0.931507
350,0.388700,0.219072,0.923077,0.894737,0.957746,0.925170
400,0.294000,0.193331,0.951049,0.944444,0.957746,0.951049
450,0.294000,0.184775,0.937063,0.907895,0.971831,0.938776
500,0.225000,0.184032,0.937063,0.897436,0.985915,0.939597


TrainOutput(global_step=550, training_loss=0.41241167415272106, metrics={'train_runtime': 91.7786, 'train_samples_per_second': 62.487, 'train_steps_per_second': 7.845, 'total_flos': 290434771553280.0, 'train_loss': 0.41241167415272106, 'epoch': 3.8194444444444446})

**Description:** This cell evaluates the trained DistilBERT model on the test set and prints out the accuracy, precision, recall, and F1 score to assess its performance on bias detection.

## 10. Save Trained Model for Deployment

Save the fine-tuned model and tokenizer to Google Drive or local storage for later use in a web application.

In [16]:
# Create a directory for the model
import os
from google.colab import files

# Setup save directory
save_dir = './bias_model'
os.makedirs(save_dir, exist_ok=True)

# Save model and tokenizer
print("Saving model and tokenizer...")
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# Save training metrics
import json
metrics_file = os.path.join(save_dir, 'training_metrics.json')
with open(metrics_file, 'w') as f:
    json.dump(trainer.state.log_history, f)

# Create zip file
!zip -r bias_model.zip {save_dir}

# Trigger download
files.download('bias_model.zip')
print("Model download should start automatically")

Saving model and tokenizer...
  adding: bias_model/ (stored 0%)
  adding: bias_model/config.json (deflated 45%)
  adding: bias_model/model.safetensors (deflated 8%)
  adding: bias_model/tokenizer_config.json (deflated 75%)
  adding: bias_model/training_metrics.json (deflated 77%)
  adding: bias_model/special_tokens_map.json (deflated 42%)
  adding: bias_model/tokenizer.json (deflated 71%)
  adding: bias_model/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model download should start automatically
